In [1]:
pip install pinecone


   ---------------------------------------- 0.0/427.3 kB ? eta -:--:--
   ------ --------------------------------- 71.7/427.3 kB 2.0 MB/s eta 0:00:01
   ------------------------ --------------- 266.2/427.3 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 427.3/427.3 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/87.5 kB ? eta -:--:--
   ---------------------------------------- 87.5/87.5 kB 4.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import pandas as pd 
import os
import time
import dotenv
dotenv.load_dotenv()

True

In [19]:
token = os.getenv("RUNPOD_TOKEN")
open_ai_base_url = os.getenv("RUNPOD_EMBEDDING_URL")
model_name = os.getenv("MODEL_NAME")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")


In [43]:
pc = Pinecone(api_key= pinecone_api_key)
em_client = OpenAI(
    api_key="rpa_YJ2IBMXDYYMJUUPNCL5QFEPD2WQKYU76P34XR5211lrda6",
    base_url="https://api.runpod.ai/v2/tyv6vmjx17v5h6/openai/v1"
)

# Try out embeddings

In [44]:
output = em_client.embeddings.create(input ="hello world", model="BAAI/bge-small-en-v1.5")
embedding = output.data[0].embedding

print(embedding)

[0.015215358696877956, -0.02272770367562771, 0.008572462946176529, -0.07437602430582047, 0.003935400862246752, 0.0027780423406511545, -0.03130016475915909, 0.0446622259914875, 0.04399107024073601, -0.007783094421029091, -0.02524453029036522, -0.033374641090631485, 0.014376416802406311, 0.046340107917785645, 0.00868686381727457, -0.0160466730594635, 0.007504718378186226, -0.019005851820111275, -0.11470626294612885, -0.01813640259206295, 0.1262989193201065, 0.029729057103395462, 0.025229275226593018, -0.0341678224503994, -0.04109290614724159, 0.006604762282222509, 0.010349494405090809, 0.02239212580025196, 0.004431139677762985, -0.12776325643062592, -0.016061928123235703, -0.020348157733678818, 0.04737734794616699, 0.011585026979446411, 0.06827463209629059, 0.007413197308778763, -0.018044881522655487, 0.040970880538225174, -0.010196959599852562, 0.02370392717421055, 0.010410508140921593, -0.02846301719546318, 0.008091977797448635, -0.015253492631018162, 0.03090357594192028, -0.0659561008

In [35]:
len(embedding)

384

In [45]:
df=pd.read_json('products/products.jsonl',lines=True)

In [46]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [47]:
 
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [49]:
df['text'].head(0)

Series([], Name: text, dtype: object)

In [52]:
texts = df['text'].tolist()

In [53]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [54]:
Merry_way_about_section

"Coffee shop Merry's Way about section: Welcome to Merry's Way Coffee, your neighborhood coffee shop located in the heart of Greenwich Village, New York City. At Merry's Way, we believe that coffee is more than just a drinkâ€”itâ€™s an experience, a moment of joy, and a way to connect with others.\n\nOur Story\nFounded in 2015, Merryâ€™s Way started as a small family-owned cafÃ© with one mission: to share the love of quality, ethically-sourced coffee with our community.\n\nMerry's passion for travel and coffee led her on a journey across South America, where she handpicked partnerships with small farms and cooperatives. We ensure that every cup we brew tells a story of dedication and care, from farm to table. Our beans are roasted in-house to bring out unique flavors that reflect the regions where they were grown.\n\nDelivery & Locations Served\nIn addition to offering a cozy place to enjoy coffee in our cafÃ©, we proudly deliver to Greenwich Village, SoHo, West Village, and Lower Manh

In [55]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

In [56]:
menue_items_text

'Menu Items: Menu Items\n\nCappuccino - $4.50\nJumbo Savory Scone - $3.25\nLatte - $4.75\nChocolate Chip Biscotti - $2.50\nEspresso shot - $2.00\nHazelnut Biscotti - $2.75\nChocolate Croissant - $3.75\nDark chocolate (Drinking Chocolate) - $5.00\nCranberry Scone - $3.50\nCroissant - $3.25\nAlmond Croissant - $4.00\nGinger Biscotti - $2.50\nOatmeal Scone - $3.25\nGinger Scone - $3.50\nChocolate syrup - $1.50\nHazelnut syrup - $1.50\nCarmel syrup - $1.50\nSugar Free Vanilla syrup - $1.50\nDark chocolate (Packaged Chocolate) - $3.00'

# Generate Embeddings

In [58]:
output = em_client.embeddings.create(input = texts,model=model_name)

In [59]:
embeddings = output.data

In [60]:
embeddings

[Embedding(embedding=[-0.014024794101715088, -0.06446216255426407, 0.003840031335130334, 0.004795746877789497, 0.030125074088573456, -0.04563017934560776, 0.03726718947291374, 0.010682650841772556, -0.027744369581341743, -0.05091046169400215, -0.052436552941799164, -0.03366560861468315, -0.0003843846498057246, -0.043768346309661865, 0.03751136362552643, -0.017183806747198105, -0.01950346864759922, -0.08063875138759613, -0.07575525343418121, -0.007935683242976665, 0.0484076663851738, -0.061226848512887955, -0.10749799013137817, -0.03693144768476486, 0.03546639904379845, -0.04852975532412529, 0.08790295571088791, -0.019823947921395302, 0.001699686050415039, -0.13087773323059082, -0.012956528924405575, 0.009797516278922558, -0.028049588203430176, -0.025790970772504807, -0.03503909334540367, 0.0009676382760517299, 0.06751435250043869, -0.075938381254673, 0.047034185379743576, 0.0005946993478573859, -0.0025638360530138016, 0.029743552207946777, -0.06415694952011108, -0.0020258883014321327, 

# Push data to database

In [61]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [63]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

# Get Closest Documents

In [65]:
output = em_client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [66]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.734850347,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the 